In [1]:
import meep as mp
import meep.adjoint as mpa
import numpy as np
from autograd import numpy as npa
from autograd import tensor_jacobian_product, grad
import nlopt
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
import math

from meep.materials import Al 

In [2]:
lambda_min = 0.4       # 최소 파장 (µm)
lambda_max = 0.7       # 최대 파장 (µm)
fmin = 1/lambda_max    # 최소 주파수
fmax = 1/lambda_min    # 최대 주파수
fcen = 0.5*(fmin+fmax) # 중앙 주파수
df = fmax-fmin         # 주파수 대역폭

resolution = 50        # 시뮬레이션 해상도
nfreq = 1             # 추출할 주파수 개수
design_region_resolution = 50

In [ ]:
tal = 1
talq3 = 0.5
tsio2 = 2

tair_near = 3.5
tair_far = 6

tpml = 0.5

tmetal_boundary = 1


design_region_width = 5       # OLED의 가로/세로 길이
design_region_height = 1
metal_boundary_height = tal+talq3+design_region_height+tsio2

al = Al

metal_boundary = mp.Medium(index=1.58)

alq3 = mp.Medium(index = 1.68)
sio2 = mp.Medium(index = 2.1)
SiN = mp.Medium(index = 1.8)
air = mp.Medium(index = 1)

Nx = int(design_region_resolution * design_region_width) + 1
Ny = int(design_region_resolution * design_region_height) + 1

boundary_layers = [mp.PML(tpml,direction=mp.X),
                   mp.PML(tpml,direction=mp.Y,side=mp.High)]

